In [1]:
# Define Julia packages
using LinearAlgebra, Symbolics

In [2]:
# Define Python packages
import numpy as np
from scipy.differentiate import jacobian

### Linear transformations as matrices - Python example

First, I want to show an example of the linear transformations fact I discussed from [slides 8-12](https://docs.google.com/presentation/d/1zZBsD1vl5pKEUobh44VGZDVOXnkF7r7LBsyj4JcBk7g/edit?usp=sharing). 

I am defining three matrices A, B, C and a vector, x. At first I am multiplying ABC which we know is a composition equivalent to writing (a(b(c(x))). After multiplying these three matrices I apply this matrix with the variable name, `composition` to the vector x. 

Then, I change the strategy and multiply the respective matrix (A, B, C) to each output of the previous composition. For example: I multiply C(x) first and assign this a new variable name. Then I multiply B by the new variable name to get *yet* another matrix with a new variable name. Lastly I multiply A by this new matrix. 

I've used an assert statement to confirm that the output of the two approaches is the same. The first approach, multiply all matrices then apply it to x is my preferred approach. I believe this is also the standard way to do it.

Because we established these approaches are the same I will use the first approach in the Jacobian examples. 

In [34]:
# Define 3 matrices
#TODO: Later convert these constants to random ints: np.random.randint(x, y)

m = 3
q = 5
p = 5
n = 4

# Set a seed 
np.random.seed(0)

A = np.random.rand(m, q)
B = np.random.rand(q, p)
C = np.random.rand(p, n)
x = np.random.rand(n, 1)

In [51]:
composition = A @ B @ C

In [48]:
composition @ x

array([[3.49606573],
       [3.8873989 ],
       [3.26430787]])

In [53]:
c_to_x = C @ x
b_to_Cx = B @ c_to_x
A_to_BCx = A @ b_to_Cx
A_to_BCx

array([[3.49606573],
       [3.8873989 ],
       [3.26430787]])

In [52]:
assert (composition @ x).all() == A_to_BCx.all()

### Example of f'(x), output of chain rule, being a product of two Jacobians - Julia example

On slide 32 of my [Introduction to Matrix Calculus](https://docs.google.com/presentation/d/1zZBsD1vl5pKEUobh44VGZDVOXnkF7r7LBsyj4JcBk7g/edit?usp=sharing) presentation I derive the chain rule on arbitrary vectory spaces. I think it's helpful to see an example of this numerically. The functions in my example below are simple enough that we can compute the derivatives by hand to get more intuition for what's happening if we need. 

Let's first talk about our desired input and output dimensions. $x$ is a vector in $R^{n}$, $h(x)$ is a function (matrix) in $R^{p}$ and $f(x) = g(h(x))$ maps from $R^{n}$ to $R^{m}$. 

In the following section I'll refer to our desired "output" dimension so let me clarify what this means in a 2x2 example. 

If I have the matrix A which is a 2 x 2 matrix:

\begin{pmatrix}
a & b\\
c & d
\end{pmatrix}

and a vector x which 2 x 1:

\begin{pmatrix} x \\ y \end{pmatrix}

The matrix-vector multiplication dimensions are 2 x 2 (m x n) @ (2 x 1) (n x 1). So, in the language I'll use below, n is the input dimension and m is the output dimension. I am calling n the input because if we think back to linear transformations as matrices where A(x) = b, x is obviously the input to the function A and its dimensions are n x 1. 

$f'(x)$ is in the output dimension $R^{m}$ so we know $f'(x)$ is an m x n matrix. We also know $h(x)$ is in $R^{p}$ and its derivative $h'(x)$ is also in $R^{p}$ so h'(x) is a p x n matrix. Here, again, p is the output dimension. Therefore it follows that if m x n is our output and $h'(x)$ is p x n then $g'(x)$ is an m x p matrix. 

Given our knowledge that these primed matrices are Jacobians we can say that $f'(x)$, a Jacobian output is the product of an m x p Jacobian and a p x n Jacobian. 

---

In the code below I am using Julia's [ForwardDiff package](https://juliadiff.org/ForwardDiff.jl/stable/#:~:text=ForwardDiff%20implements%20methods%20to%20take,mode%20automatic%20differentiation%20(AD).). The first function `z(x)` takes looks like the following in vector-valued function notation:

\begin{pmatrix}
x_{1}^{2} + x_{2}\\
x_{1}x_{2}
\end{pmatrix}

and the second function `w(y)` is:

\begin{pmatrix}
sin_{y1} + y_{2}\\
y_{1}y_{2}
\end{pmatrix}

Then, after defining x (the point we'd like to take partial derivatives with respect to) and evaluating z(x) at this point I begin taking Jacobians. The reader can refer to slide 27 in [Introduction to Matrix Calculus](https://docs.google.com/presentation/d/1zZBsD1vl5pKEUobh44VGZDVOXnkF7r7LBsyj4JcBk7g/edit?usp=sharing) to recall how to do these by hand and confirm the answers. 

In [33]:
using Pkg
Pkg.add("ForwardDiff")

   Resolving package versions...
   Installed CommonSubexpressions ─ v0.3.1
   Installed ForwardDiff ────────── v1.0.1
   Installed DiffResults ────────── v1.1.0
    Updating `~/.julia/environments/v1.11/Project.toml`
  [f6369f11] + ForwardDiff v1.0.1
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [bbf7d656] + CommonSubexpressions v0.3.1
  [163ba53b] + DiffResults v1.1.0
  [f6369f11] + ForwardDiff v1.0.1
Precompiling project...
   1166.9 ms  ✓ CommonSubexpressions
    747.5 ms  ✓ DiffResults
   2264.7 ms  ✓ ForwardDiff
    616.5 ms  ✓ RecursiveArrayTools → RecursiveArrayToolsForwardDiffExt
   1009.1 ms  ✓ ForwardDiff → ForwardDiffStaticArraysExt
   2340.5 ms  ✓ Symbolics → SymbolicsForwardDiffExt
  6 dependencies successfully precompiled in 7 seconds. 178 already precompiled.


In [34]:
using ForwardDiff

In [36]:
function z(x)
    return [x[1]^2 + x[2]
        x[1] * x[2]]
end

z (generic function with 1 method)

In [37]:
function w(y)
    return [sin(y[1]) + y[2],
    y[1] * y[2]]
end

w (generic function with 1 method)

In [38]:
x = [1.0, 2.0]

2-element Vector{Float64}:
 1.0
 2.0

In [39]:
zx = z(x)

2-element Vector{Float64}:
 3.0
 2.0

In [40]:
Dhx = ForwardDiff.jacobian(z, x)

2×2 Matrix{Float64}:
 2.0  1.0
 2.0  1.0

In [41]:
Dghx = ForwardDiff.jacobian(w, zx)

2×2 Matrix{Float64}:
 -0.989992  1.0
  2.0       3.0

In [42]:
J_composed = Dghx * Dhx

2×2 Matrix{Float64}:
  0.020015  0.0100075
 10.0       5.0

### Symbolic Jacobians in Symbolic Kronecker Product notation - Julia

The goal of this section is to develop a notation to write the Jacobian without having to write the Jacobian. This notation is called the Kronecker product. 

You may be asking: "Why would we want to do this?" It seems there are two reasons for this. First, it's simpler. As we will see we can write the Jacobian as a sum of two Kronecker products without needing to think about partial derivatives at all. Second, the Jacobian is someitmes computationally inefficient (the Kronecker product is too) and we'll want to adopt a different way to compute this. 

In order to understand the Kronecker product we will need to first understand matrix "vectorization." Let's also define and see an example of the Kronecker product. 

The first code block below defines a 2 x 2 matrix X. These dimensions (m x n) are important because the dimensions of our vectorized matrix will be mn x 1. Vectorizing a matrix stacks the columns on top of each other.

Next, let's define the Kronecker product. 

> If A is an m x n matrix with entries $a_{ij}$ and B is a p x q matrix then their Kronecker product A ⊗ B is an mp x nq matrix. The Kronecker product takes each entry of A and multiplies it by the matrix B s.t the resultant matrix has "all combinations of A's entries with the matrix B."

The example in the fourth code block below shows the definition clearly. The $X_{11}$ entry, **a** is multiplied by every letter in Z and this populates a mini 2x2 in `kron(X, Z)`. Perhaps one way to think about the Kronecker product, using the definition, is as a scalar multiplication of each $A_{ij}$ entry with B s.t. the result is an appropriately sized matrix in the Kronecker product output. The numerical example `kron(T, U)` shows this scalar multiplication perspective. 

In [43]:
# Define variables
@variables a, b, c, d

X = [a b; c d]

2×2 Matrix{Num}:
 a  b
 c  d

In [44]:
# Vec the 2x2 matrix -> 4 x 1 vector

vec(X)

4-element Vector{Num}:
 a
 c
 b
 d

In [45]:
# First symbolic example of Kronecker products
@variables l, m, n, o

Z = [l m; n o]

2×2 Matrix{Num}:
 l  m
 n  o

In [46]:
kron(X, Z)

4×4 Matrix{Num}:
 a*l  a*m  b*l  b*m
 a*n  a*o  b*n  b*o
 c*l  c*m  d*l  d*m
 c*n  c*o  d*n  d*o

In [49]:
T = [1 2; 3 6]

2×2 Matrix{Int64}:
 1  2
 3  6

In [50]:
U = [10 20; 30 40]

2×2 Matrix{Int64}:
 10  20
 30  40

In [51]:
kron(T, U)

4×4 Matrix{Int64}:
 10   20   20   40
 30   40   60   80
 30   60   60  120
 90  120  180  240

In [18]:
X^2

2×2 Matrix{Num}:
 a^2 + b*c  a*b + b*d
 a*c + c*d  b*c + d^2

In [19]:
# Take the Jacobian of Y, vector valued function, with respect to parameters in X?

jac(Y, X) = Symbolics.jacobian(vec(Y), vec(X))

jac (generic function with 1 method)

In [20]:
# I think we would get the same answer if we took the Jacobian by hand. Right, the partial derivatives...
# ... of each term in X^2 are in the first row. 1,1 entry of X^2 is the first row of J
# ... and 2,1 entry of X^2 is the second row of J, etc. 

J = jac(X^2, X)

4×4 Matrix{Num}:
 2a      b      c   0
  c  a + d      0   c
  b      0  a + d   b
  0      b      c  2d

In [6]:
begin 
    I2 = [1 0; 0 1]
    kron(I2,X) + kron(X', I2)
end

4×4 Matrix{Num}:
 2a      b      c   0
  c  a + d      0   c
  b      0  a + d   b
  0      b      c  2d

In [7]:
# Symbolic representation
SymB = [a b; c d]

kron(I2, SymB)

4×4 Matrix{Num}:
 a  b  0  0
 c  d  0  0
 0  0  a  b
 0  0  c  d

In [8]:
B = rand(2, 2)

2×2 Matrix{Float64}:
 0.737501  0.0859048
 0.705054  0.575519

In [9]:
kron(I2, B)

4×4 Matrix{Float64}:
 0.737501  0.0859048  0.0       0.0
 0.705054  0.575519   0.0       0.0
 0.0       0.0        0.737501  0.0859048
 0.0       0.0        0.705054  0.575519

In [10]:
# A kron I
@variables c1 , c2, c3, c4

A = [a b; c d]
C = [c1; c2; c3; c4]
vC = vec(C)
kron(A, I2) * vC

4-element Vector{Num}:
 a*c1 + b*c3
 a*c2 + b*c4
 c*c1 + c3*d
 c*c2 + c4*d

### Quick examples of two properties of trace and *why* these hold

In [11]:
@variables e, f, g, h

B = [e f; g h]

2×2 Matrix{Num}:
 e  f
 g  h

In [32]:
tr(A*B)

a*e + b*g + c*f + d*h

In [33]:
A*B

2×2 Matrix{Num}:
 a*e + b*g  a*f + b*h
 c*e + d*g  c*f + d*h

In [35]:
B*A

2×2 Matrix{Num}:
 a*e + c*f  b*e + d*f
 a*g + c*h  b*g + d*h

In [36]:
tr(B*A)

a*e + b*g + c*f + d*h

In [38]:
@variables i, j, k, l

C = [i j; k l]

2×2 Matrix{Num}:
 i  j
 k  l

In [39]:
tr(A*B*C)

(a*e + b*g)*i + (a*f + b*h)*k + (c*e + d*g)*j + (c*f + d*h)*l

In [43]:
tr(C*A*B)

(a*i + c*j)*e + (a*k + c*l)*f + (b*i + d*j)*g + (b*k + d*l)*h

In [40]:
tr(B*C*A)

a*(e*i + f*k) + b*(g*i + h*k) + c*(e*j + f*l) + d*(g*j + h*l)

### Show the matrix dot product is equal to tr(A^T* B)


This confirms our intuition about the element-wise multiplication equaling the tr(A^T) * B.

*Would be nice to explain more about the trace operator and some of the properties like cyclic property.* What is my hypothesis on why the trace operator appears so often. 

Look at this [link](https://math.stackexchange.com/questions/4453933/why-is-the-trace-of-a-matrix-important).